In [ ]:
import pandas as pd
from glob import glob

In [ ]:
main = pd.read_csv("../dataset/combined.csv")

In [ ]:
comm = set(main["community"].to_list())

In [ ]:
print(len(comm))

In [ ]:

for c in comm:
    if c == "other":
        dff = main.query("community == 'other'")
        dff = dff.drop(["community"], axis=1)
        dff.to_csv("./dataset/other.csv", index=False)
        continue
    dff = main.query("community == \'{}\'".format(c))
    dff = dff.drop(["community"], axis=1)
    dff.to_csv("./dataset/{}.csv".format(c), index=False)
    # break


# add edge component

In [ ]:
all_nodes = [i for i in range(7015)]

In [ ]:
edgeOrdering = [0]*7015

In [ ]:
files = glob("dataset\\*")
files.pop() # drop the other option

In [ ]:
from collections import Counter

In [ ]:
for file in files:
    df = pd.read_csv(file)
    nodes = df['node1'].to_list()
    count = dict(Counter(nodes))
    ordering = []
    for i in count:
        ordering.append([count[i], i])
    ordering.sort(reverse=True)
    c = 0
    for pair in ordering:
        edgeOrdering[pair[1]] = c
        c += 1
    # break

In [ ]:
print(edgeOrdering)

# temporal ordering

In [ ]:
temporalOrdering = [0]*7015

In [ ]:
for file in files[:]:
    df = pd.read_csv(file)
    df = df.sort_values(by=['temporal_similarity', 'topic_similarity'], ascending=False)
    d = set()
    c = 0
    for node in df['node1'].to_list():
        if node not in d:
            temporalOrdering[node] = c
            c += 1
            d.add(node)
    # break

In [ ]:
print(temporalOrdering)

# temporal ordering

In [ ]:
topicOrdering = [0]*7015

In [ ]:
for file in files[:]:
    df = pd.read_csv(file)
    df = df.sort_values(by=['topic_similarity', 'temporal_similarity'], ascending=False)
    d = set()
    c = 0
    for node in df['node1'].to_list():
        if node not in d:
            topicOrdering[node] = c
            c += 1
            d.add(node)
    # break

In [ ]:
print(topicOrdering)

In [ ]:
df = pd.DataFrame(data={"node": all_nodes,
                        "edgeOrdering": edgeOrdering,
                        "topicOrdering": topicOrdering,
                        "temporalOrdering": temporalOrdering})

In [ ]:
df.to_csv("seriation.csv", index=False)

# community within it

In [ ]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
communityOrdering = [0]*7015

In [ ]:
kool = 1
for file in files:
    print(kool)
    kool+=1
    G = nx.Graph()
    dff = pd.read_csv(file)
    # G.add_nodes_from(set(dff['node1'].to_list()))
    for ind in dff.index:
        G.add_edge(dff["node1"][ind], dff["node2"][ind])
    
    partition = nx.community.louvain_communities(G)
    # print(partition)
    c = 0
    for part in partition:
        for i in part:
            communityOrdering[i] = c
            c += 1

In [ ]:
df["communityOrdering"] = communityOrdering
df.to_csv("seriation.csv", index=False)